<a href="https://colab.research.google.com/github/arpagon/podcast-coffeebreak/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detect GPU

In [ ]:
!nvidia-smi

# Install CondaColab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

# Check CondaColab it's ok

In [ ]:
import condacolab
condacolab.check()

# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

/opt/arpagon/mambaforge/envs/OpenAIWhisper/lib/python3.10/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


KeyError: 'CLOUDSDK_CONFIG'

In [ ]:
podcast_audio_directory = '/content/drive/MyDrive/AI/podcast/audio'
podcast_text_directory = '/content/drive/MyDrive/AI/podcast/text'
podcast_stats_csv = '/content/drive/MyDrive/AI/podcast/stats.csv'
whisper_model = 'medium'
decode_options = {"language": "es"}

In [ ]:
!mkdir -p {podcast_audio_directory}
!mkdir -p {podcast_text_directory}

# Clone Repo

In [ ]:
!git clone https://github.com/arpagon/podcast-coffeebreak.git

# Install my env

In [ ]:
!mamba env update -n base -f /content/podcast-coffeebreak/environment.yml

# Check youtube-dl Version

In [ ]:
!youtube-dl --version

# Download Podcast

In [ ]:
!bash /content/podcast-coffeebreak/utils/000_download_podcast.sh -u 'https://podcasts.google.com/feed/aHR0cDovL2ZlZWRzLmZlZWRidXJuZXIuY29tL1BvZGNhc3RDb2ZmZWVCcmVhaw?sa=X&ved=0CD4Q9sEGahcKEwjw1Zv4i8_6AhUAAAAAHQAAAAAQEw' -d /content/drive/MyDrive/AI/podcast/audio

In [1]:
import os
import whisper
import json
import time
import pandas as pd


df = pd.DataFrame(columns=
        [
            'id', 
            'audio_filename',
            'text_filename',
            'size_in_bytes',
            'size_in_sconds',
            'transcribe_status',
            'transcribe_start',
            'transcribe_end',
            'transcribe_duration',
            'transcribe_model',
            'transcribe_language'
        ], index=['id']
)
model = whisper.load_model(whisper_model)

for filename in os.listdir(podcast_audio_directory):
    id=filename.split('.')[0]
    audio_filename = os.path.join(podcast_audio_directory, filename)
    text_filename = os.path.join(podcast_text_directory, filename.split('.')[0] + '.json')
    size_in_bytes = os.path.getsize(audio_filename)
    # size_in_sconds=audio.info.length
    df.loc[id]={
        "id": id,
        "audio_filename": audio_filename,
        "text_filename": text_filename,
        "size_in_bytes": size_in_bytes,
        # "size_in_sconds": size_in_sconds
    }
    if not os.path.exists(text_filename):
        if os.path.isfile(audio_filename):
            transcribe_start = time.time()
            print('\r transcribing {0} at {1}'.format(audio_filename, transcribe_start))
            result = model.transcribe(audio_filename, decode_options)
            transcribe_end = time.time()
            transcribe_duration = transcribe_start - transcribe_end
            transcribe_status = "1"
            transcribe_model=model
            with open(text_filename, 'w',encoding='utf8') as text_filename_file:
                print('\r Writing {0}'.format(text_filename))
                json.dump(result, text_filename_file, indent=4)
            print('\r end transcribe {0} at {1} duration of {2} sec'.format(audio_filename, transcribe_end, transcribe_duration))
            df.loc[id]={
                "transcribe_status": transcribe_status,
                "transcribe_start": transcribe_start,
                "transcribe_end": transcribe_end,
                "transcribe_duration": transcribe_duration,
                "transcribe_model": whisper_model,
                "transcribe_language": decode_options["language"]
            }
    df.to_csv(podcast_stats_csv, mode='a', index=False, header=False)
    print("\r {0} Data appended successfully".format(id))

SyntaxError: invalid syntax (2718330330.py, line 46)